한 꿀벌 군집으로 부터 6km내에 존재하는 꿀벌 군집만 추출

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

def find_nearby_rows(df, distance_threshold=6):
    """
    BallTree를 이용하여 각 행의 위도 및 경도를 비교하고, 
    지정된 거리 이내에 있는 다른 행들의 ID를 찾아 반환하는 최적화 함수.
    
    Args:
        df (pd.DataFrame): Latitude, Longitude, ID 열이 포함된 데이터프레임
        distance_threshold (float): 거리 임계값 (단위: km)

    Returns:
        pd.DataFrame: 각 행에 대해 인근 군집들의 ID를 담은 데이터프레임
    """
    # 위도, 경도를 라디안으로 변환
    coords = np.radians(df[['Latitude', 'Longitude']].values)
    tree = BallTree(coords, metric='haversine')
    radius = distance_threshold / 6371.0  # km -> radians

    # 반경 내 인덱스 검색
    indices = tree.query_radius(coords, r=radius)

    nearby_ids = []
    for i, neighbors in enumerate(indices):
        # 자기 자신 제외
        filtered = [df.iloc[j]['ID'] for j in neighbors if j != i]
        nearby_ids.append(filtered)

    # 결과 DataFrame 구성
    result_df = pd.DataFrame(index=df.index)
    for idx, ids in enumerate(nearby_ids):
        for n, nearby_id in enumerate(ids):
            result_df.loc[idx, f'nearby_id_{n+1}'] = nearby_id
    return result_df

In [2]:
# 입력데이터
df = pd.read_csv("../../data/final/4_Apis_cerana.csv")
df1 = pd.read_csv("../../data/final/4_Apis_mellifera.csv")

result = find_nearby_rows(df)
print(result)

result1 = find_nearby_rows(df1)
print(result1)

      nearby_id_1  nearby_id_2  nearby_id_3  nearby_id_4  nearby_id_5  \
0           718.0        718.0        718.0       5621.0        718.0   
1          6692.0          NaN          NaN          NaN          NaN   
2           648.0        648.0        648.0        648.0        648.0   
3          2743.0       2743.0       2743.0       2743.0       2743.0   
4           718.0        718.0        718.0        718.0        718.0   
...           ...          ...          ...          ...          ...   
1154       4529.0          NaN          NaN          NaN          NaN   
1155          NaN          NaN          NaN          NaN          NaN   
1156        648.0        648.0        648.0        648.0        648.0   
1157          NaN          NaN          NaN          NaN          NaN   
1158        718.0        718.0        718.0        718.0       6692.0   

      nearby_id_6  nearby_id_7  nearby_id_8  nearby_id_9  nearby_id_10  ...  \
0           718.0        718.0          NaN 

In [3]:
result.to_csv("../../data/processed/6_6km_nearby_cer.csv")
result1.to_csv("../../data/processed/6_6km_nearby_mel.csv")